In [2]:
from tqdm.notebook import tqdm
from setup import setup
setup()

In [17]:
import datasets

from torchvision import transforms

imagenet = datasets.load_dataset("ILSVRC/imagenet-1k", trust_remote_code=True)
print(imagenet)

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1281167
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 50000
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 100000
    })
})


In [6]:
from src.supcl.util import TwoCropTransform
from torchvision import transforms

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
normalize = transforms.Normalize(mean=mean, std=std)

train_transform = transforms.Compose(
    [
        transforms.RandomResizedCrop(size=32, scale=(0.2, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(0.4, 0.4, 0.4, 0.1)], p=0.8),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        normalize,
    ]
)

In [9]:
from src.supcl.util import load_imagenet_hf
imagenet_train = load_imagenet_hf(object(), TwoCropTransform(train_transform))['train']
imagenet_train[0]

{'image': [tensor([[[ 0.2282,  0.2796,  0.3138,  ...,  0.5193,  0.5364,  0.5707],
           [ 0.2796,  0.3309,  0.3138,  ...,  0.6392,  0.6563,  0.6049],
           [ 0.3481,  0.3481,  0.3481,  ...,  0.7419,  0.7077,  0.1939],
           ...,
           [ 0.9132,  0.9132,  0.9132,  ...,  0.9132,  0.9303,  0.9303],
           [ 0.9303,  0.9303,  0.9303,  ...,  0.8961,  0.8789,  0.8961],
           [ 0.9303,  0.9474,  0.9474,  ...,  0.9132,  0.9132,  0.9132]],
  
          [[ 0.4328,  0.4678,  0.4678,  ...,  0.7304,  0.7304,  0.7304],
           [ 0.4678,  0.4853,  0.5028,  ...,  0.8354,  0.8529,  0.7304],
           [ 0.5028,  0.5028,  0.5028,  ...,  0.9230,  0.8354, -0.0574],
           ...,
           [ 1.0630,  1.0630,  1.0630,  ...,  1.0280,  1.0280,  1.0280],
           [ 1.0455,  1.0455,  1.0630,  ...,  1.0105,  0.9755,  0.9930],
           [ 1.0455,  1.0455,  1.0805,  ...,  1.0455,  1.0455,  1.0630]],
  
          [[ 0.6879,  0.7054,  0.7054,  ...,  0.8622,  0.8448,  0.8274],
  

In [2]:
dataset['train']

Dataset({
    features: ['image', 'label'],
    num_rows: 1281167
})

In [10]:
from torchvision import datasets as torchvision_datasets
from src.supcl.util import TwoCropTransform
cifar10_dataset = torchvision_datasets.CIFAR10(
    root='../datasets/', transform=TwoCropTransform(train_transform), download=True
)

Files already downloaded and verified


In [20]:
cifar10_dataset[0]

([tensor([[[-1.0219, -1.1589, -1.1418,  ..., -0.1143, -0.1143, -0.0972],
           [-1.3987, -1.5699, -1.4672,  ..., -0.3712, -0.3883, -0.4568],
           [-1.3815, -1.5014, -1.2959,  ..., -0.3198, -0.4226, -0.5253],
           ...,
           [ 0.3652,  0.0912, -0.0801,  ..., -0.2513,  0.2624,  0.2453],
           [ 0.3823,  0.1426, -0.0116,  ..., -0.5253,  0.0741, -0.0287],
           [ 0.2967,  0.1254,  0.0741,  ..., -1.0904, -0.3369, -0.1486]],
  
          [[-0.9153, -1.0553, -1.0378,  ...,  0.0126,  0.0126,  0.0301],
           [-1.3004, -1.4755, -1.3704,  ..., -0.2500, -0.2675, -0.3375],
           [-1.2829, -1.4055, -1.1954,  ..., -0.1975, -0.3025, -0.4076],
           ...,
           [ 0.5028,  0.2227,  0.0476,  ..., -0.1275,  0.3978,  0.3803],
           [ 0.5203,  0.2752,  0.1176,  ..., -0.4076,  0.2052,  0.1001],
           [ 0.4328,  0.2577,  0.2052,  ..., -0.9853, -0.2150, -0.0224]],
  
          [[-0.6890, -0.8284, -0.8110,  ...,  0.2348,  0.2348,  0.2522],
           